In [8]:
import pandas as pd

file_path = "15.01 Tr. 6 Export for Patryk Kusztal 45648.csv"

# Ponownie wczytam plik, analizując jego zawartość, aby znaleźć odpowiedni wiersz nagłówków
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Znalezienie wiersza nagłówka (pierwszy wiersz zawierający "Velocity" jako kolumnę)
header_line = next(i for i, line in enumerate(lines) if "Velocity" in line)

# Wczytanie pliku od linii nagłówka
df = pd.read_csv(file_path, skiprows=header_line, sep=';', decimal=',')

# Podgląd pierwszych wierszy, aby sprawdzić poprawność wczytania
df.head()



,Timestamp,Seconds,Velocity,Acceleration,Odometer,Latitude,Longitude,Heart Rate,Player Load,Positional Quality (%),HDOP,#Sats
0,15.01.2025 18:05,0.0,1.42,-0.122217,0.00,36.889434,31.051139,0,0.0,70.5,0.78,14
1,15.01.2025 18:05,0.1,1.94,0.087716,0.12,36.889435,31.051140,0,0.0,72.0,0.78,14
2,15.01.2025 18:05,0.2,2.40,0.328711,0.24,36.889435,31.051141,0,0.0,69.9,0.78,14
3,15.01.2025 18:05,0.3,2.78,0.558277,0.34,36.889436,31.051142,0,0.0,71.0,0.78,14
4,15.01.2025 18:05,0.4,3.07,0.745226,0.49,36.889437,31.051142,0,0.1,71.4,0.78,14


In [19]:
# Obliczenie średniej ruchomej (każda obserwacja + 4 kolejne wartości)
df['Velocity_SMA'] = df['Velocity'].rolling(window=5, min_periods=1).mean()
df['Acceleration_SMA'] = df['Acceleration'].rolling(window=5, min_periods=1).mean()

# Grupowanie danych według przedziałów prędkości i wybór 5 największych przyspieszeń
bins = range(0, int(df['Velocity_SMA'].max()) + 2)  # Przedziały 0-1, 1-2, ..., max_velocity+1
df['Velocity_Bin'] = pd.cut(df['Velocity_SMA'], bins=bins, right=False)

# Znalezienie 5 największych wartości przyspieszenia dla każdego przedziału prędkości
top_acceleration_by_bin = df.groupby('Velocity_Bin').apply(
    lambda x: x.nlargest(5, 'Acceleration_SMA')[['Timestamp','Seconds', 'Velocity_SMA', 'Acceleration_SMA']]
).reset_index(drop=True)


top_acceleration_by_bin.to_excel('wyniki.xlsx')

C:\Users\00tim351\AppData\Local\Temp\ipykernel_7252\2726668530.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  top_acceleration_by_bin = df.groupby('Velocity_Bin').apply(
C:\Users\00tim351\AppData\Local\Temp\ipykernel_7252\2726668530.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_acceleration_by_bin = df.groupby('Velocity_Bin').apply(
